<a href="https://colab.research.google.com/github/YuhaiW/00/blob/main/DCGAN7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter 
 

In [9]:
from torch.nn.modules.activation import LeakyReLU
class Discriminator(nn.Module):
  def __init__(self, channel_img, features_d):
    super(Discriminator, self).__init__()
    # input = N * channel_img * 64 * 64
    self.disc = nn.Sequential(
        nn.Conv2d(
            channel_img,
            features_d,
            kernel_size = 4,
            stride = 2,
            padding = 1
        ), # 32 * 32
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d*2, 4, 2, 1),  # 16 *16
        self._block(features_d*2, features_d*4, 4, 2, 1), # 8 * 8
        self._block(features_d*4, features_d*8, 4, 2, 1), # 4 * 4
        nn.Conv2d(features_d*8, 1, 4, 2, 0),  #1 * 1
        nn.Sigmoid(),
    )





  def _block(self, in_channel, out_channel, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels = in_channel, 
                  out_channels = out_channel, 
                  kernel_size = kernel_size,
                  stride = stride,
                  padding = padding,
                  bias = False), # cause we will use "BatchNorm2d", so the bias = False
        nn.BatchNorm2d(num_features = out_channel),
        nn.LeakyReLU(0.2),
    )

  def forward(self, x):
    return self.disc(x)

In [10]:
from torch.nn.modules.conv import ConvTranspose2d
class Generator(nn.Module):
  def __init__(self, z_dim, channels_img, feature_g):
    super(Generator, self).__init__()
    self.gen = nn.Sequential(
    self._block(z_dim, feature_g*16, 4, 1, 0),
    self._block(z_dim*16, feature_g*8, 4, 2, 1),
    self._block(z_dim*8, feature_g*4, 4, 2, 1),
    self._block(z_dim*4, feature_g*2, 4, 2, 1),
    nn,ConvTranspose2d(feature_g*2, channels_img, 4, 2, 1),
    nn.Tanh(), #[-1,1]
    )


  def _block(self, in_channel, out_channel, kernel_size, stride, padding):
    return nn.Sequential(
         nn.ConvTranspose2d(in_channels = in_channel, 
                  out_channels = out_channel, 
                  kernel_size = kernel_size,
                  stride = stride,
                  padding = padding,
                  bias = False), # cause we will use "BatchNorm2d", so the bias = False
        nn.BatchNorm2d(num_features = out_channel),
        nn.LeakyReLU(0.2),
    )
  
def forward(self, x):
    return self.gen(x)

In [11]:
def initialize_weight(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.02)
      